# GoProLensFront
https://github.com/vjann/DLC

### Description:
Front view of gopro with the lens. One video, 2.7k at 120 fps, of uninjured mouse

## Create a new project

Keep the projects separate. This function creates a new project with subdirectories and a basic configuration file under DeepLabCut directory.
You can add new videos to the project at any stage of the project. 

In [1]:
import datetime
import deeplabcut
from time import sleep

In [7]:
task='goprolensfront' # Enter the name of your experiment Task
experimenter='vj' # Enter the name of the experimenter
video=[r"C:\Users\vjj14\Documents\GoProLens\Not_Injured_Straight_2.7k_120fps.MP4"] # Enter the paths of your videos you want to grab frames from.
path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory= r'C:\Users\vjj14\Desktop\DeepLabCut', copy_videos=False) #change the working directory to where you want the folders created.
# The function returns the path, where your project is. 

Created "C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos"
Created "C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\labeled-data"
Created "C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\training-datasets"
Created "C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models"
Creating the symbolic link of the video
Created the symlink of C:\Users\vjj14\Documents\GoProLens\Not_Injured_Straight_2.7k_120fps.MP4 to C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\Not_Injured_Straight_2.7k_120fps.MP4
C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\Not_Injured_Straight_2.7k_120fps.MP4
Generated "C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\config.yaml"

A new project with name goprolensfront-vj-2019-06-25 is created at C:\Users\vjj14\Desktop\DeepLabCut and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's n

In [2]:
path_config_file=r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\config.yaml"
project_name='goprolensfront'
def write_log(text, new_line=True):
    time_log= open("{0}log{1}.txt".format(path_config_file[:-11], project_name),"a+")
    if new_line:
        time_log.write("\n")
    time_log.write(text)
    time_log.close()
    print("log written")
    
def prompt_with_timeout(prompt_message):
  print('Waiting... please press Ctrl-C when you wish to proceed.')
  try:
    for i in range(0, 2*60): # 2 minutes
      sleep(1)
    return "No Input Received"
  except KeyboardInterrupt:
    return input(prompt_message)

## Extract frames from videos 
Select diverse frames, which are typical for the behavior you study that should be labeled.
(algo=='uniform') selects N frames either uniformly sampled from a particular video (or folder) . Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.
Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later, crop the frames to remove unnecessary parts.
Always check the output of cropping.

In [41]:
?deeplabcut.add_new_videos

In [12]:
for x in range(1, 5):
    deeplabcut.add_new_videos(path_config_file, [r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\split{}.mp4".format(x)])

New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.
New video was added to the project! Use the function 'extract_frames' to select frames for labeling.


In [13]:
start = datetime.datetime.now()
write_log(str(start) + "***extract_frames start")

%matplotlib inline
deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 

end = datetime.datetime.now()
write_log(str(end) + "***extract_frames end")
write_log("   time elapsed:" + str(end-start))
write_log("     " + prompt_with_timeout("number of frames selected?") + "frames labeled")

log written
Config file read successfully.
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Documents\GoProLens\Not_Injured_Straight_2.7k_120fps.MP4 ?
yes/nono
Do you want to extract (perhaps additional) frames for video: C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\Not_Injured_Straight_2.7k_120fps_Trim_284.mp4 ?
yes/nono


KeyboardInterrupt: 

In [19]:
?deeplabcut.extract_frames

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [5]:
start = datetime.datetime.now()
write_log(str(start) + "***label_frames start")

%gui wx
deeplabcut.label_frames(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***label_frames end")
write_log("   time elapsed:" + str(end-start))

log written


ERROR:root:Cannot activate multiple GUI eventloops


You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.
log written
log written


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. 

In [6]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by vj.
They are stored in the following folder: C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\labeled-data\Not_Injured_Straight_2.7k_120fps_labeled.


SystemError: <built-in method write of _io.BufferedWriter object at 0x000001EA27905888> returned a result with an error set

If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

In [7]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

log written
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!
log written
log written


In [6]:
?deeplabcut.create_training_dataset

## Start training - If you want to use a CPU, continue. 
### If you want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
start = datetime.datetime.now()
write_log(str(start) + "***train_network start")
write_log("   DESCRIPTION:" + prompt_with_timeout("Training description(Network, Iterations, etc) "))

deeplabcut.train_network(path_config_file, saveiters=1000, displayiters=100, maxiters=60002)

end = datetime.datetime.now()
write_log(str(end) + "***train_network end")
write_log("   time elapsed:" + str(end-start))

log written
Waiting... please press Ctrl-C when you wish to proceed.
Training description(Network, Iterations, etc) retrain network with addition of training labels
log written


Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['LeftEar',
                      'RightEar',
                      'Nose',
                      'OtherHand',
                      'DomHand',
                      'Index',
                      'Middle',
                      'Ring',
                      'Pinky',
                      'Pellet'],
 'batch_size': 4,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_goprolensfrontJun25\\goprolensfront_vj90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref

INFO:tensorflow:Restoring parameters from C:\Users\vjj14\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\.conda\envs\dlc-windowsGPU\lib\site-packages\deeplabcut\pose_estimation_tensorflow\models\pretrained\resnet_v1_50.ckpt


Max_iters overwritten as 60002
Display_iters overwritten as 100
Save_iters overwritten as 1000
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': 'C:\\Users\\vjj14\\Desktop\\DeepLabCut\\goprolensfront-vj-2019-06-25\\dlc-models\\iteration-0\\goprolensfrontJun25-trainset90shuffle1\\train\\snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth

iteration: 100 loss: 0.0672 lr: 0.005
iteration: 200 loss: 0.0152 lr: 0.005
iteration: 300 loss: 0.0141 lr: 0.005
iteration: 400 loss: 0.0137 lr: 0.005
iteration: 500 loss: 0.0129 lr: 0.005
iteration: 600 loss: 0.0121 lr: 0.005
iteration: 700 loss: 0.0114 lr: 0.005
iteration: 800 loss: 0.0095 lr: 0.005
iteration: 900 loss: 0.0100 lr: 0.005
iteration: 1000 loss: 0.0095 lr: 0.005
iteration: 1100 loss: 0.0108 lr: 0.005
iteration: 1200 loss: 0.0083 lr: 0.005
iteration: 1300 loss: 0.0091 lr: 0.005
iteration: 1400 loss: 0.0080 lr: 0.005
iteration: 1500 loss: 0.0083 lr: 0.005
iteration: 1600 loss: 0.0083 lr: 0.005
iteration: 1700 loss: 0.0077 lr: 0.005
iteration: 1800 loss: 0.0086 lr: 0.005
iteration: 1900 loss: 0.0076 lr: 0.005
iteration: 2000 loss: 0.0080 lr: 0.005
iteration: 2100 loss: 0.0074 lr: 0.005
iteration: 2200 loss: 0.0078 lr: 0.005
iteration: 2300 loss: 0.0071 lr: 0.005
iteration: 2400 loss: 0.0071 lr: 0.005
iteration: 2500 loss: 0.0066 lr: 0.005
iteration: 2600 loss: 0.0076 lr: 0

iteration: 21100 loss: 0.0031 lr: 0.02
iteration: 21200 loss: 0.0027 lr: 0.02


In [18]:
?deeplabcut.train_network

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [9]:
start = datetime.datetime.now()
write_log(str(start) + "***evaluate_network start")

deeplabcut.evaluate_network(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***evaluate_network end")
write_log("   time elapsed:" + str(end-start))

log written


Config:
{'all_joints': [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9]],
 'all_joints_names': ['LeftEar',
                      'RightEar',
                      'Nose',
                      'OtherHand',
                      'DomHand',
                      'Index',
                      'Middle',
                      'Ring',
                      'Pinky',
                      'Pellet'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets\\iteration-0\\UnaugmentedDataSet_goprolensfrontJun25\\goprolensfront_vj90shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': 'C:\\Users\\vjj14\\.conda\\envs\\dlc-windowsGPU\\lib\\site-packages\\deeplabcut\\pose_estimation_tensorflow\\models\\pretrained\\resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref

Running  DeepCut_resnet50_goprolensfrontJun25shuffle1_60000  with # of trainingiterations: 60000
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1\train\snapshot-60000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1\train\snapshot-60000


Analyzing data...


136it [00:44,  3.23it/s]


Done and results stored for snapshot:  snapshot-60000
Results for 60000  training iterations: 90 1 train error: 12.81 pixels. Test error: 30.2  pixels.
With pcutoff of 0.1  train error: 12.08 pixels. Test error: 21.94 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)
log written
log written


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [6]:
videos=[[r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\Not_Injured_Straight_2.7k_120fps_Trim_284.mp4"]]
videos=[[r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\2.7k-6-26.mp4"]]
for videofile_path in videos:
    start = datetime.datetime.now()
    write_log(str(start) + "***analyze_videos start")
    write_log("   VIDEO: " + str(videofile_path))

    deeplabcut.analyze_videos(path_config_file,videofile_path, destfolder=r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos")

    end = datetime.datetime.now()
    write_log(str(end) + "***analyze_videos end")
    write_log("   time elapsed:" + str(end-start))

log written
log written
Using snapshot-60000 for model C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1
INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1\train\snapshot-60000


INFO:tensorflow:Restoring parameters from C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1\train\snapshot-60000


Starting to analyze %  C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\2.7k-6-26.mp4
Loading  C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\2.7k-6-26.mp4
Duration of video [s]:  78.19 , recorded with  119.88 fps!
Overall # of frames:  9374  found with (before cropping) frame dimensions:  2704 1520
Starting to extract posture


9393it [24:04,  6.39it/s]                                                           

Detected frames:  9374


Saving results in C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!
log written
log written


In [10]:
?deeplabcut.analyze_videos

## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [38]:
alg = 'jump'
param = 0 #epsilon or p_bound
videofile_path = [r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\Not_Injured_Straight_2.7k_120fps_Trim_284.mp4"]

start = datetime.datetime.now()
write_log(str(start) + "***extract_outlier_frames start")

deeplabcut.extract_outlier_frames(path_config_file, videofile_path, outlieralgorithm=alg, epsilon=param, extractionalgorithm='uniform')

num_frames = prompt_with_timeout("number of frames selected?")
write_log("   VIDEOS: {0}, ALGORITHM: {1}, PARAMETER: {2}, NUM FRAMES: {3}".format(str(videofile_path), alg, param, num_frames))
end = datetime.datetime.now()
write_log(str(end) + "***extract_outlier_frames start")
write_log("   time elapsed:" + str(end-start))

log written
network parameters: DeepCut_resnet50_goprolensfrontJun25shuffle1_60000
Method  jump  found  7171  putative outlier frames.
Do you want to proceed with extracting  200  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.
yes/noyes
Loading video...
Duration of video [s]:  59.826433333333334 , recorded @  119.88011988011988 fps!
Overall # of frames:  7172 with (cropped) frame dimensions: 
Uniformly extracting of frames from 0.0  seconds to 59.83  seconds.
Let's select frames indices: [835, 5042, 4318, 2596, 662, 496, 215, 265, 4746, 601, 5499, 6720, 4751, 5097, 4875, 5300, 2484, 5278, 2827, 3590, 3098, 2156, 5511, 5067, 1326, 6914, 3189, 35, 5678, 55, 4990, 2260, 1898, 3222, 3103, 3637, 1869, 313, 4343, 927, 1876, 3378, 5976, 7167, 2084, 4790, 6675, 1643, 6398, 1646, 3940, 1120, 6225, 1528, 6933, 787, 3401, 6463, 4279, 2608, 3204, 4325, 5647, 790, 5312, 4774, 3838, 5699, 1273, 2229

In [33]:
?deeplabcut.extract_outlier_frames

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [4]:
start = datetime.datetime.now()
write_log(str(start) + "***refine_frames start")

%gui wx
deeplabcut.refine_labels(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***refine_frames end")
write_log("   time elapsed:" + str(end-start))


ERROR:root:Cannot activate multiple GUI eventloops


Windows
Checking labels if they are outside the image
A training dataset file is already found for this video. The refined machine labels are merged to this data!
Closing... The refined labels are stored in a subdirectory under labeled-data. Use the function 'merge_datasets' to augment the training dataset, and then re-train a network using create_training_dataset followed by train_network!


In [17]:
?deeplabcut.refine_labels

In [5]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
start = datetime.datetime.now()
write_log(str(start) + "***merge_datasets start")

deeplabcut.merge_datasets(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***merge_datasets end")
write_log("   time elapsed:" + str(end-start))

Merged data sets and updated refinement iteration to 2.
Now you can create a new training set for the expanded annotated images (use create_training_dataset).


## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [6]:
start = datetime.datetime.now()
write_log(str(start) + "***create_training_dataset start")

deeplabcut.create_training_dataset(path_config_file)

end = datetime.datetime.now()
write_log(str(end) + "***create_training_dataset end")
write_log("   time elapsed:" + str(end-start))

log written
C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\training-datasets\iteration-0\UnaugmentedDataSet_goprolensfrontJun25  already exists!
C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\labeled-data\Not_Injured_Straight_2.7k_120fps_Trim_284/CollectedData_vj.h5  not found (perhaps not annotated)
C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1  already exists!
C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1//train  already exists!
C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\dlc-models\iteration-0\goprolensfrontJun25-trainset90shuffle1//test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!
log written
log written


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [8]:
for videofile_path in videos:
    start = datetime.datetime.now()
    write_log(str(start) + "***create_labeled_video start")
    write_log("   VIDEOS: {}".format(str(videofile_path)))

    deeplabcut.create_labeled_video(path_config_file,videofile_path)

    end = datetime.datetime.now()
    write_log(str(end) + "***create_labeled_video end")
    write_log("   time elapsed:" + str(end-start))

log written
log written
Starting %  C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos ['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\goprolensfront-vj-2019-06-25\\videos\\2.7k-6-26.mp4']
Loading  C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\2.7k-6-26.mp4 and data.
False 0 2704 0 1520
9374
Duration of video [s]:  78.19 , recorded with  119.88 fps!
Overall # of frames:  9374 with cropped frame dimensions:  2704 1520
Generating frames and creating video.


100%|███████████████████████████████████████████| 9374/9374 [04:50<00:00, 32.23it/s]


log written
log written


In [22]:
?deeplabcut.create_labeled_video

## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [45]:
videofile_path

['C:\\Users\\vjj14\\Desktop\\DeepLabCut\\frontslowmo-vj-2019-06-05\\videos\\1080p.MOV']

In [ ]:
%matplotlib notebook 
#for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)

In [58]:
?deeplabcut.plot_trajectories

In [5]:
import numpy as np
import pandas as pd

## Pandas analysis of Data

In [6]:
df = pd.read_hdf(r"C:\Users\vjj14\Desktop\DeepLabCut\goprolensfront-vj-2019-06-25\videos\Not_Injured_Straight_2.7k_120fpsDeepCut_resnet50_goprolensfrontJun25shuffle1_60000.h5")

In [7]:
df.describe()

scorer    DeepCut_resnet50_goprolensfrontJun25shuffle1_60000              \
bodyparts                                            LeftEar               
coords                                                     x           y   
count                                             105.000000  105.000000   
mean                                             1503.574034  282.822988   
std                                               123.504190  148.876787   
min                                              1115.445024    2.103334   
25%                                              1513.385615  179.197799   
50%                                              1525.802026  356.329814   
75%                                              1574.840784  376.584492   
max                                              1603.957578  591.656752   

scorer                                                                   \
bodyparts                 RightEar                                 Nose   
coords     likelihood            x           y  likelihood            x   
count      105.000000   105.000000  105.000000  105.000000   105.000000   
mean         0.754422  1228.926231  295.152503    0.648425  1279.542788   
std          0.349366   112.345263  128.470838    0.389366    67.997671   
min          0.011810  1121.069498   17.644119    0.013904  1112.686656   
25%          0.702394  1167.111786  236.726774    0.229473  1298.300790   
50%          0.948059  1187.847504  306.673974    0.935322  1306.858853   
75%          0.972724  1252.416697  380.099058    0.975977  1312.839120   
max          0.989698  1596.719448  592.694887    0.997744  1329.079322   

scorer                                             ...                  \
bodyparts                            OtherHand     ...          Middle   
coords              y  likelihood            x     ...      likelihood   
count      105.000000  105.000000   105.000000     ...      105.000000   
mean       541.612881    0.757608  1470.581010     ...        0.212546   
std        294.659570    0.303802    23.870181     ...        0.216159   
min          0.467965    0.083271  1397.795978     ...        0.009539   
25%        361.810872    0.438936  1474.633912     ...        0.043747   
50%        683.666083    0.940182  1477.511329     ...        0.116635   
75%        781.840202    0.989868  1487.194695     ...        0.348191   
max        843.296909    0.998812  1502.352868     ...        0.928926   

scorer                                                                   \
bodyparts         Ring                                Pinky               
coords               x           y  likelihood            x           y   
count       105.000000  105.000000  105.000000   105.000000  105.000000   
mean       1203.465760  527.344406    0.179768  1188.847454  481.378264   
std          85.309552  276.284686    0.206569    85.621317  210.663198   
min        1106.416076    3.670274    0.015309  1083.233552    2.336228   
25%        1134.323946  312.720063    0.038509  1131.790052  351.560631   
50%        1156.636322  643.046982    0.082039  1165.272328  483.720958   
75%        1306.713865  750.132088    0.215790  1229.804114  682.661053   
max        1512.598078  847.781815    0.716293  1471.553391  737.614871   

scorer                                                      
bodyparts                   Pellet                          
coords     likelihood            x           y  likelihood  
count      105.000000   105.000000  105.000000  105.000000  
mean         0.320001  1360.870127  401.357341    0.026652  
std          0.303094   158.169530  215.119825    0.011568  
min          0.014774  1115.942986    2.058006    0.008609  
25%          0.049929  1244.838982  332.161833    0.018764  
50%          0.243046  1312.281226  381.894572    0.023159  
75%          0.604282  1515.836557  483.282255    0.031656  
max          0.928717  1596.482531  844.483600    0.070943  

[8 rows x 30 columns]